<a id='indice'></a>

## Índice
### [Librerias](#librerias)
### [Carga de datos](#carga)
### [Pre-proceso](#pre)
   1. ### [Variables cualitativas](#cuali)
   2. ### [Valores nulos](#null)
   3. ### [Variable Y (ESTADO_BAT)](#y)
   4. ### [Conjunto train-test](#train)
   5. ### [SMOTE](#smote)
   6. ### [Optimizar los parámetros](#optimizar)
   7. ### [Seleccionar la métrica y los diferentes modelos que se quieren probar](#metrica_modelos)
   8. ### [Métrica Brier score multiclase](#brier_score)


### [Modelos](#modelos)
   - ### [Árboles de Clasificación](#arbol)
   - ### [Regresión Logística](#regresion)
   - ### [K-vecinos más cercanos](#KNN)
   - ### [Máquinas de vectores soporte](#SVC)
   - ### [Redes neuronales](#NN)
   - ### [Bagging](#Bagging)
   - ### [Boosting](#Boosting)
   - ### [Stacking](#Stacking)

### Librerias <a id='librerias'></a>

Las librerias utilizadas han sido las siguientes:

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import seaborn as sb

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn import tree

# Preprocesado
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import statsmodels.api as sm

# Optimización de parámetros
# ------------------------------------------------------------------------------
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import f1_score, make_scorer

# Modelado
# ------------------------------------------------------------------------------
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.model_selection import cross_val_score

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

### Carga de datos  <a id='carga'></a>

Cargamos los datos que utilizaremos para realizar el proyecto. 

In [ ]:
import pandas as pd
df = pd.read_excel('dataset_escalado.xlsx')
df.head()

### Pre-proceso <a id='pre'></a>

#### 1. Eliminamos las variables cualitativas. <a id='cuali'></a>

In [3]:
df = df.drop(['ID_PDA','SN_PDA','FECHA_FAB','FECHA_TEST','SN_BAT'], axis=1)

#### 2. Esta función me permite eliminar de la base de datos los valores que son NaN y los Inf, en caso de que los haya. <a id='null'></a>

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
clean_dataset(df)

#### 3. Definimos el conjunto de variables X, junto con la variable objetivo Y (ESTADO_BAT). <a id='y'></a>

In [6]:
X = df.drop(['ESTADO_BAT'], axis=1).values
y = df['ESTADO_BAT'].values

Creamos un histograma para observar la distribución de las clases presentes en la variable objetivo. 

In [ ]:
sb.catplot('ESTADO_BAT',data=df,kind="count")

#### 4. División de los datos en un conjunto de train y test. <a id='train'></a>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                        df.drop(columns = 'ESTADO_BAT'),
                                        df['ESTADO_BAT'],
                                        random_state = 123,
                                        stratify = df['ESTADO_BAT'],
                                        train_size=0.7)

#### 5. Aplicamos el algoritmo SMOTE a los datos de train, para solucionar el problema del desbalanceo de clases. <a id='smote'></a>

In [9]:
sm = SMOTE(sampling_strategy='auto', k_neighbors=2, random_state=100)
X_train, y_train = sm.fit_resample(X_train, y_train)

#### 6. Optimizamos los parámetros. A continuación, definimos una función para elegir los parámetros óptimos de los diferentes modelos probados. <a id='optimizar'></a>

In [10]:
def clfGrid(X, y, clasificadores, n_clas):
    res = {}
    for clf_cnt, clf in enumerate(clasificadores):
        clf.fit(X, y)     
        res[n_clas[clf_cnt]] = {}
        res[n_clas[clf_cnt]]["F1"] = clf.best_score_
        res[n_clas[clf_cnt]]["Mejor clasificador"] = clf.best_estimator_
    return res

#### 7. Seleccionar la métrica con la que maximizar los parámetros (en este caso f1_weighted) y los diferentes modelos que se quieren probar.  <a id='metrica_modelos'></a>

En cada caso se elige el modelo y los valores o rango de valores que queremos probar en cada parámetro.

In [11]:
f1 = make_scorer(f1_score , average='weighted')

clasificadores = [GridSearchCV(estimator=LogisticRegression(), 
                               param_grid = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                                             'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'), 
                                             'max_iter' : [100, 200, 300]}, scoring=f1),
                  
                  GridSearchCV(estimator=AdaBoostClassifier(), 
                               param_grid = {'n_estimators':[50,100,200,500]}, 
                               scoring=f1),
                  
                  GridSearchCV(estimator=LinearSVC(), 
                               param_grid = {'C':[0.1,1,1.5]}, scoring=f1),
                  
                  GridSearchCV(estimator=KNeighborsClassifier(), 
                               param_grid = {'n_neighbors':[1,2,3,4,5,6], 
                                             'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
                                             'p':[1,2]}, scoring=f1),
                  
                  GridSearchCV(estimator=DecisionTreeClassifier(), 
                               param_grid = {'criterion':['gini','entropy'],
                                             'splitter':['random','best'],
                                             'min_samples_split':[2,4,6], 
                                             'max_depth': [2, 5, 10, 15, 20], 
                                             'min_samples_leaf': [5, 4, 3, 2, 1]}, 
                               scoring=f1),
                  
                 GridSearchCV(estimator = MLPClassifier(random_state = 1), 
                              param_grid={'activation': ('identity', 'logistic', 'tanh', 'relu'), 
                                          'solver': ('lbfgs', 'sgd', 'adam'), 'max_iter': [2, 5, 10]}, 
                              scoring=f1),
                  
                 GridSearchCV(estimator = SVC(random_state = 1), 
                              param_grid = dict(C = [1,2,4], gamma = ['scale', 'auto'], 
                                                kernel = ['poly', 'rbf', 'sigmoid']), 
                              scoring=f1)]

n_clas = ['Regressión logística', 'AdaBoost', 'SVC Lineal','Vecinos','Arboles', 'Perceptrón Multicapa', 'SVC']

In [ ]:
clfGrid(X_train,y_train, clasificadores, n_clas)

Una vez obtenidos los parámetros óptimos de los diferentes clasificadores, pasaremos a aplicar cada modelo con sus respectivos parámetros.

### Desarrollo de la métrica f1_score:

El F1-score, combina las medidas de precisón y recall (exhaustividad) lo que nos hace medir el rendimiento de un modelo de forma más fiable.

Este valor oscila entre los valores 0 y 1. Siendo 1 el mejor valor pues indicaría que el modelo predice con 100% de fiabilidad. Por otro lado, el valor 0 indicaría que el modelo no se asemeja en nada a la realidad.

**Precisión:**

El número de valores de la clase positiva bien predichos dividido por el número de valores de la clase positiva bien predichos más el número de valores de la clase positiva mal predichos.

$precision=\frac{\text{TP}}{\text{TP} + \text{FP}}$

Indicando la calidad del modelo, pues muestra el acierto en la clasificación de los valores de la clase positiva.

**Recall:**

El número de valores de la clase positiva bien predichos dividido por el número de valores de la clase positiva bien predichos más el número de valores de la clase negativa mal predichos.

$recall=\frac{\text{TP}}{\text{TP} + \text{FN}}$

Indicando la cantidad de que el modelo es capaz de predecir bien, pues muestra la proporción de valores clasificados como clase positiva frente a la realidad.

**Fórmula F1_score:**

$F_{1}=\frac{\text{Precisión} \times \text{Exhaustividad} \times \text{2}}{\text{Precisión} + \text{Exhaustividad}}$


#### 8. En caso de desbalanceo en las clases en la variable respuesta, utilizamos la métrica Brier score multiclase.  <a id='brier_score'></a>

In [13]:
def brier_multi(targets, probs):
    return np.mean(np.sum((probs - targets)**2, axis=1))

### Desarrollo de la métrica Brier Score:

El Brier Score es una métrica que calcula el error de predicción basándose en la fiabilidad del clasificador.

Sus valores oscilan entre 0 y 1. El 0 indica que no ha habido error, por tanto es el mejor caso. Por el contrario, si se da un número alto como el 1 indicaría que se ha fallado en todas las predicciones.

Debido a su cálculo en las  predicciones unidimensionales, es estrictamente equivalente al error cuadrático medio aplicado a las probabilidades predichas.

**Fórmula Brier Score:**

$\frac{1}{N} \sum_{t=1} ^{N} {(f_t - o_t)^2}$

### Modelos <a id='modelos'></a>

#### Modelo de Árboles de Clasificación  <a id='arbol'></a>

In [ ]:
# Creamos el modelo
modelo = DecisionTreeClassifier(max_depth = 10, splitter="random",random_state=10)
# Validación cruzada para ver los resultados del modelo
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
# Entrenamos el modelo
fit = modelo.fit(X_train,y_train)
# Predecimos los datos de test
pred = modelo.predict(X_test)
# Calculamos la probabilidad que el modelo asigna a cada observación de pertenecer a cada clase
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El Brier Score da:',brier_multi(probs,bien))
print(f'El f1 de predict da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 cross-val da(Decision Tree): {f1:.5f}')

#### Modelo de Regresión Logística <a id='regresion'></a>

In [ ]:
# Creamos el modelo
modelo =  LogisticRegression(C=1, solver='liblinear')
# Validación cruzada para ver los resultados del modelo
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
# Entrenamos el modelo
fit = modelo.fit(X_train,y_train)
# Predecimos los datos de test
pred = modelo.predict(X_test)
# Calculamos la probabilidad que el modelo asigna a cada observación de pertenecer a cada clase
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (Regresión Logística): {f1:.5f}')

#### Modelo de K-Vecinos más cercanos <a id='KNN'></a>

In [ ]:
# Creamos el modelo
modelo =  KNeighborsClassifier(n_neighbors=1, p=1)
# Validación cruzada para ver los resultados del modelo
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
# Entrenamos el modelo
fit = modelo.fit(X_train,y_train)
# Predecimos los datos de test
pred = modelo.predict(X_test)
# Calculamos la probabilidad que el modelo asigna a cada observación de pertenecer a cada clase
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (KNN): {f1:.5f}')

#### Modelo de Máquinas de Vectores Soporte <a id='SVC'></a>

In [ ]:
# Creamos el modelo
modelo =  SVC(C=4, gamma='auto', random_state=1,probability=True)
# Validación cruzada para ver los resultados del modelo
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
# Entrenamos el modelo
fit = modelo.fit(X_train,y_train)
# Predecimos los datos de test
pred = modelo.predict(X_test)
# Calculamos la probabilidad que el modelo asigna a cada observación de pertenecer a cada clase
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (SVC): {f1:.5f}')

#### Redes neuronales <a id='NN'></a>

In [ ]:
# Creamos el modelo
modelo =  MLPClassifier(random_state=1, max_iter=1000)
# Validación cruzada para ver los resultados del modelo
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
# Entrenamos el modelo
fit = modelo.fit(X_train,y_train)
# Predecimos los datos de test
pred = modelo.predict(X_test)
# Calculamos la probabilidad que el modelo asigna a cada observación de pertenecer a cada clase
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (NN): {f1:.5f}')

#### Probamos Bagging con los diferentes clasificadores  <a id='Bagging'></a>

In [ ]:
# Creamos el modelo
modelo = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=1, p=1),n_estimators=200)
# Validación cruzada para ver los resultados del modelo
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
# Entrenamos el modelo
fit = modelo.fit(X_train,y_train)
# Predecimos los datos de test
pred = modelo.predict(X_test)
# Calculamos la probabilidad que el modelo asigna a cada observación de pertenecer a cada clase
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (Bagging): {f1:.5f}')

#### Probamos Boosting con los diferentes clasificadores <a id='Boosting'></a>

In [ ]:
# Creamos el modelo
modelo=AdaBoostClassifier(base_estimator=LogisticRegression(C=0.1, max_iter=100, penalty='l2',
                            random_state=10, solver='lbfgs'),n_estimators=200, algorithm='SAMME')
# Validación cruzada para ver los resultados del modelo
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
# Entrenamos el modelo
fit = modelo.fit(X_train,y_train)
# Predecimos los datos de test
pred = modelo.predict(X_test)
# Calculamos la probabilidad que el modelo asigna a cada observación de pertenecer a cada clase
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (Boosting): {f1:.5f}')

#### Probamos Stacking con los diferentes clasificadores <a id='Stacking'></a>

In [ ]:
# Elegimos los dos clasificadores con los que crearemos el modelo de Stacking
estimators=[('mlp',MLPClassifier(random_state=1, max_iter=1000)),('LinearSVC', LinearSVC(C=0.1, max_iter=10000))]
# Creamos el modelo
modelo=StackingClassifier(estimators=estimators,final_estimator=KNeighborsClassifier(n_neighbors=1, p=1))
# Validación cruzada para ver los resultados del modelo
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
# Entrenamos el modelo
fit = modelo.fit(X_train,y_train)
# Predecimos los datos de test
pred = modelo.predict(X_test)
# Calculamos la probabilidad que el modelo asigna a cada observación de pertenecer a cada clase
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (Stacking): {f1:.5f}')